In [1]:
import cv2 as cv
import matplotlib.pyplot as plt
import numpy as np
import time
import os

In [2]:
def findOutliers(arr):

	lower, med, upper = np.quantile(arr, [0.25, 0.5, 0.75])

	range = upper - lower
	min = lower - 1.5*range
	max = upper + 1.5*range

	"""print(f"L outlier: {min}")
	print(f"L        : {lower}")
	print(f"Median   : {med}")
	print(f"U        : {upper}")
	print(f"U outlier: {max}")"""

	# Find outliers
	outliers = np.where((arr < min) | (arr > max))[0]
		
	return list(outliers)

In [5]:
def processImage(filename, imageType, debug = True):
	finalImage = cv.imread(filename, cv.IMREAD_COLOR)[:, :, ::-1]

	# Change image type if needed
	if (imageType == 'hsv'):
		finalImage = cv.cvtColor(finalImage, cv.COLOR_RGB2HSV)

	# data
	data = [[] for _ in range(finalImage.shape[2])]

	# Sort the data
	start = time.time()
	for i in range(finalImage.shape[2]):
		data[i] = finalImage[:, :, i:(i + 1)]
		for _ in range(finalImage.shape[2] - 1):
			data[i] = data[i].flatten()
	if (debug == True):
		print(f"It took {time.time() - start:.3f} seconds to sort the data")

	# Find outliers
	if (debug == True):
		start = time.time()
	indexes = []
	for i in range(finalImage.shape[2]):
		badValues = findOutliers(data[i])
		if (len(badValues) > 0):
			indexes.extend(badValues)
	outliers = set(indexes)
	outliers = sorted(list(outliers), reverse = True)
	if (debug == True):
		print(f"It took {time.time() - start:.3f} seconds to find outliers")

	# Remove Outliers
	if (debug == True):
		start = time.time()
	for i in range(finalImage.shape[2]):
		data[i] = np.delete(data[i], outliers)
	if (debug == True):
		print(f"It took {time.time() - start:.3f} seconds to remove outliers")

	# Display the data
	if (debug == True):
		for i in range(finalImage.shape[2]):
			print(f"{imageType[i].upper()} Min: {np.min(data[i])}")
			print(f"{imageType[i].upper()} Avg: {np.sum(data[i]) / len(data[i]):.1f}")
			print(f"{imageType[i].upper()} Med: {np.quantile(data[i], 0.5)}")
			print(f"{imageType[i].upper()} Max: {np.max(data[i])}")
			print()
	return data

In [10]:
data = [[] for _ in range(3)]

for filename in os.listdir('Images/Color-Calibration-JPG/Test-Set'):
    f = os.path.join('Images/Color-Calibration-JPG/Test-Set', filename)
    # checking if it is a file
    if os.path.isfile(f):
        if (os.path.splitext(f)[1] == '.jpg'):
            info = processImage(f, 'hsv', False)
            for i in range(len(info)):
                data[i].extend(info[i])

for i, color in enumerate(['R', 'G', 'B']):
    print(f"{color} Min: {np.min(data[i])}")
    print(f"{color} Avg: {np.sum(data[i]) / len(data[i]):.1f}")
    print(f"{color} Med: {np.quantile(data[i], 0.5)}")
    print(f"{color} Max: {np.max(data[i])}")
    print()

R Min: 30
R Avg: 31.3
R Med: 31.0
R Max: 33

G Min: 121
G Avg: 159.0
G Med: 161.0
G Max: 196

B Min: 126
B Avg: 183.5
B Med: 179.0
B Max: 254

